In [1]:
pip install langchain faiss-cpu transformers sentence-transformers flask


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 37.8 MB/s eta 0:00:00


In [3]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [5]:
import pandas as pd

dataframe = pd.read_csv("/content/Medicine_Details.csv")
dataframe.head()

,Medicine Name,Composition,Uses,Side_effects,Image URL,Manufacturer,Excellent Review %,Average Review %,Poor Review %
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Roche Products India Pvt Ltd,22,56,22
1,Augmentin 625 Duo Tablet,Amoxycillin (500mg) + Clavulanic Acid (125mg),Treatment of Bacterial infections,Vomiting Nausea Diarrhea Mucocutaneous candidi...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glaxo SmithKline Pharmaceuticals Ltd,47,35,18
2,Azithral 500 Tablet,Azithromycin (500mg),Treatment of Bacterial infections,Nausea Abdominal pain Diarrhea,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Alembic Pharmaceuticals Ltd,39,40,21
3,Ascoril LS Syrup,Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...,Treatment of Cough with mucus,Nausea Vomiting Diarrhea Upset stomach Stomach...,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Glenmark Pharmaceuticals Ltd,24,41,35
4,Aciloc 150 Tablet,Ranitidine (150mg),Treatment of Gastroesophageal reflux disease (...,Headache Diarrhea Gastrointestinal disturbance,"https://onemg.gumlet.io/l_watermark_346,w_480,...",Cadila Pharmaceuticals Ltd,34,37,29


In [6]:
dataframe.columns

Index(['Medicine Name', 'Composition', 'Uses', 'Side_effects', 'Image URL',
       'Manufacturer', 'Excellent Review %', 'Average Review %',
       'Poor Review %'],
      dtype='object')

In [7]:
# Import required libraries
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from flask import Flask, request, jsonify
import threading


In [9]:
from google.colab import files
uploaded = files.upload()

def load_and_process_dataset(filepath):
    df = pd.read_csv(filepath)
    relevant_columns = ['Medicine Name', 'Composition', 'Uses', 'Side_effects']
    df = df[relevant_columns]
    df['text'] = df.apply(
        lambda row: f"Medicine: {row['Medicine Name']}\n"
                    f"Composition: {row['Composition']}\n"
                    f"Uses: {row['Uses']}\n"
                    f"Side Effects: {row['Side_effects']}\n",
        axis=1
    )
    return df['text'].tolist()

dataset_path = list(uploaded.keys())[0]
texts = load_and_process_dataset(dataset_path)


Saving Medicine_Details.csv to Medicine_Details (1).csv


In [10]:
def split_texts(texts):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = splitter.create_documents(texts)
    return documents

documents = split_texts(texts)


In [11]:
def create_vector_store(documents, embedding_model="sentence-transformers/all-MiniLM-L6-v2"):
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
    vector_store = FAISS.from_documents(documents, embeddings)
    vector_store.save_local("faiss_index")
    return vector_store

vector_store = create_vector_store(documents)


<ipython-input-11-eff4a993e77c>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# Load the FAISS index with explicit deserialization permission
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
vector_store = FAISS.load_local(
    "faiss_index",
    HuggingFaceEmbeddings(model_name=embedding_model),
    allow_dangerous_deserialization=True
)

# Create retriever
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})


In [14]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


generation_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(pipeline=generation_pipeline)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

<ipython-input-14-2344a3edf738>:7: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generation_pipeline)


In [17]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

# Define a prompt template
retrieval_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="You are a helpful assistant. Use the following context to answer the question:\n\n{context}\n\nQuestion: {question}\nAnswer:"
)

# Create a RetrievalQA chain
rag_pipeline = RetrievalQA.from_chain_type(
    retriever=retriever,
    chain_type="stuff",
    llm=llm,
    return_source_documents=True,
    chain_type_kwargs={"prompt": retrieval_prompt},
)


In [19]:
# Simulate a query to the RAG pipeline
query = "What are the uses of paracetamol?"
response = rag_pipeline(query)

# Extract the answer and sources
answer = response['result']
sources = [doc.page_content for doc in response["source_documents"]]

# Display the result
print(f"Query: {query}")
print(f"Answer: {answer}")
print("Sources:")
for i, source in enumerate(sources, start=1):
    print(f"{i}. {source}")


<ipython-input-19-a4c36c99b0a9>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = rag_pipeline(query)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Query: What are the uses of paracetamol?
Answer: Pain reliefTreatment of Fever
Sources:
1. Medicine: Paracip Infusion
Composition: Paracetamol (1000mg)
Uses:  Pain reliefTreatment of Fever
Side Effects: Headache Insomnia difficulty in sleeping Constipation Itching Anxiety Stomach pain Nausea Vomiting
2. Medicine: Medomol Drop
Composition: Paracetamol (100mg/ml)
Uses:  Pain reliefTreatment of Fever
Side Effects: Indigestion Stomach pain Nausea Vomiting
3. Medicine: Dolodart 120mg/5ml Suspension
Composition: Paracetamol (120mg/5ml)
Uses:  Pain reliefTreatment of Fever
Side Effects: Indigestion Stomach pain Nausea Vomiting


In [20]:
# Function to query the RAG pipeline
def test_rag_pipeline(rag_pipeline, query):
    """
    Test the RAG pipeline with a given query and display the results.

    Parameters:
        rag_pipeline: The RAG pipeline instance.
        query (str): The query to pass to the pipeline.

    Returns:
        None
    """
    print(f"Running query: {query}")

    # Query the RAG pipeline
    response = rag_pipeline(query)

    # Extract answer and source documents
    answer = response['result']
    sources = [doc.page_content for doc in response.get("source_documents", [])]

    # Print results
    print(f"\nQuery: {query}")
    print(f"Answer: {answer}")
    print("\nSources:")
    if sources:
        for i, source in enumerate(sources, start=1):
            print(f"{i}. {source}")
    else:
        print("No sources found.")

# Example queries to test
queries = [
    "What are the uses of paracetamol?",
    "What are the side effects of ibuprofen?",
    "Who manufactures aspirin?",
]

# Test each query
for query in queries:
    test_rag_pipeline(rag_pipeline, query)
    print("\n" + "-" * 50 + "\n")


Running query: What are the uses of paracetamol?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



Query: What are the uses of paracetamol?
Answer: Pain reliefTreatment of Fever

Sources:
1. Medicine: Paracip Infusion
Composition: Paracetamol (1000mg)
Uses:  Pain reliefTreatment of Fever
Side Effects: Headache Insomnia difficulty in sleeping Constipation Itching Anxiety Stomach pain Nausea Vomiting
2. Medicine: Medomol Drop
Composition: Paracetamol (100mg/ml)
Uses:  Pain reliefTreatment of Fever
Side Effects: Indigestion Stomach pain Nausea Vomiting
3. Medicine: Dolodart 120mg/5ml Suspension
Composition: Paracetamol (120mg/5ml)
Uses:  Pain reliefTreatment of Fever
Side Effects: Indigestion Stomach pain Nausea Vomiting

--------------------------------------------------

Running query: What are the side effects of ibuprofen?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



Query: What are the side effects of ibuprofen?
Answer: Nausea Vomiting Stomach pain Indigestion Heartburn

Sources:
1. Medicine: Ibukind Plus 100 mg/162.5 mg Suspension
Composition: Ibuprofen (100mg) + Paracetamol (162.5mg)
Uses:  Pain reliefTreatment of Fever
Side Effects: Nausea Vomiting Abdominal pain Heartburn Diarrhea
2. Medicine: Ibucon Plus Suspension
Composition: Ibuprofen (100mg) + Paracetamol (162.5mg)
Uses:  Pain reliefTreatment of Fever
Side Effects: Nausea Vomiting Abdominal pain Heartburn Diarrhea
3. Medicine: Hifenac-D  Tablet
Composition: Aceclofenac (100mg) + Paracetamol (325mg) + Serratiopeptidase (15mg)
Uses:  Pain relief
Side Effects: Nausea Vomiting Stomach pain Indigestion Heartburn Loss of appetite Diarrhea

--------------------------------------------------

Running query: Who manufactures aspirin?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



Query: Who manufactures aspirin?
Answer: Preva-AS

Sources:
1. Medicine: Preva-AS 150 Tablet
Composition: Aspirin (150mg) + Clopidogrel (75mg)
Uses:  Prevention of heart attack and stroke
Side Effects: Abdominal pain Bruise Diarrhea Gastrointestinal bleeding Increased bleeding tendency Nosebleeds
2. Medicine: Preva-AS 75 Tablet
Composition: Aspirin (75mg) + Clopidogrel (75mg)
Uses:  Prevention of heart attack and stroke
Side Effects: Abdominal pain Bruise Diarrhea Gastrointestinal bleeding Increased bleeding tendency Nosebleeds
3. Medicine: Aspisol 75 Tablet
Composition: Aspirin (75mg) + Glycine (37.5mg)
Uses:  Prevention of heart attack and stroke
Side Effects: Increased bleeding tendency Indigestion Diarrhea

--------------------------------------------------



In [22]:
import os

# Check if the faiss_index directory exists
if os.path.exists('/content/faiss_index'):
    print("Directory exists")
else:
    print("Directory not found!")


Directory exists


In [23]:
import shutil

shutil.make_archive('/content/faiss_index', 'zip', '/content/faiss_index')

'/content/faiss_index.zip'

In [24]:
# Import necessary libraries
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Load the vector store (ensure the FAISS index and embeddings are set up correctly)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Define the embedding model
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

# Load the FAISS index
vector_store = FAISS.load_local(
    folder_path="faiss_index",
    embeddings=HuggingFaceEmbeddings(model_name=embedding_model),
    allow_dangerous_deserialization=True  # Ensure this is safe for your setup
)

# Create a retriever
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Load the pre-trained LLM
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Initialize the LLM
llm_pipeline = pipeline("text-generation", model="google/flan-t5-base", device=-1)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Define the RAG pipeline
rag_pipeline = RetrievalQA.from_chain_type(
    retriever=retriever,
    llm=llm,
    return_source_documents=True
)

# Function to query the RAG pipeline
def query_rag_pipeline(query):
    """
    Query the RAG pipeline and return the result.

    Args:
        query (str): The user's query.

    Returns:
        dict: A dictionary containing the query, answer, and sources.
    """
    print(f"\nQuery: {query}")

    # Perform the query
    response = rag_pipeline(query)

    # Extract answer and source documents
    answer = response.get('result', "No answer found.")
    sources = response.get("source_documents", [])

    # Display results
    print(f"Answer: {answer}")
    print("\nSources:")
    if sources:
        for i, source in enumerate(sources, start=1):
            print(f"{i}. {source.page_content}")
    else:
        print("No sources found.")

    return {
        "query": query,
        "answer": answer,
        "sources": [doc.page_content for doc in sources]
    }

# Example usage
if __name__ == "__main__":
    print("Welcome to PharmaQA.ai!")
    print("Type your queries to retrieve answers, or type 'exit' to quit.\n")

    while True:
        user_query = input("Enter your query: ")
        if user_query.lower() == "exit":
            print("Goodbye!")
            break
        result = query_rag_pipeline(user_query)





The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartFo

Welcome to PharmaQA.ai!
Type your queries to retrieve answers, or type 'exit' to quit.

Enter your query: What are the side effects of ibuprofen?

Query: What are the side effects of ibuprofen?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Medicine: Ibukind Plus 100 mg/162.5 mg Suspension
Composition: Ibuprofen (100mg) + Paracetamol (162.5mg)
Uses:  Pain reliefTreatment of Fever
Side Effects: Nausea Vomiting Abdominal pain Heartburn Diarrhea

Medicine: Ibucon Plus Suspension
Composition: Ibuprofen (100mg) + Paracetamol (162.5mg)
Uses:  Pain reliefTreatment of Fever
Side Effects: Nausea Vomiting Abdominal pain Heartburn Diarrhea

Medicine: Hifenac-D  Tablet
Composition: Aceclofenac (100mg) + Paracetamol (325mg) + Serratiopeptidase (15mg)
Uses:  Pain relief
Side Effects: Nausea Vomiting Stomach pain Indigestion Heartburn Loss of appetite Diarrhea

Question: What are the side effects of ibuprofen?
Helpful Answer:

Sources:
1. Medicine: Ibukind Plus 100 mg/162.5 mg Suspension
Composition: Ibuprofen (100mg) + Paracetamol (162.5mg)
Uses:  Pain reliefTreatmen